In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
!pip install mlflow
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)


    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479513631630414

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [25]:
#everything inside this with statement will be saved as history, like we can keep changing 
#values of alpha in lasso and each time results will be tracked in history

with mlflow.start_run():

    #create tag that will be associated to the current run like name of the developer 
    #if the project is done with a big team so that we can find runs by a specific person
    mlflow.set_tag("developer", "ajeet")  

    
    #log_params includes hyperparameters of the model but also any extra information that we consider relevant
    #can we put in it because they do not have any effect on the model's performance
    
    #for instance, for us its important to know which data was used for training and validation so 
    #we are putting their path in th log_params
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    
    alpha = 0.001   #playing with different alpha
    mlflow.log_param("alpha", alpha)   #also log the parameter alpha
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    #We will also keep track of performance of model so let us log the rmse
    mlflow.log_metric("rmse", rmse)

    #save the model
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [14]:
!pip  install xgboost
import xgboost as xgb

In [20]:
!pip install hyperopt   #hyperopt is a library that uses some bayesian methods to try to find the best set of hyperparameters
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials #fmin fun that we will use to minimize the objective fn
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
#first of all we need to define the objective fn, here params is set of hyperparameters xgboot for this specific run
def objective(params):
    with mlflow.start_run():
        
        #log the model name and its hyperparameters
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')], 
            early_stopping_rounds=50 #if 50 or more iterations w/o any improvement on validaton set then optimization of xgboost will stop!
        )
        
        #once the model is trained, make predictions on validation set
        y_pred = booster.predict(valid)
        
        #calculate rmse on validation set
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        #log the rmse
        mlflow.log_metric("rmse", rmse)

    #return the loss, hyperopt will optimize the model by minimizing this metric and also send signal okay to hyperopt that optimization was successful
    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
#the range of diff. hyperparameters which we want hyperopt to explore them

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), #hp.quniform('max_depth', 4, 100, 1)) will give a float b/w 4 and 100 and then we convert it to int
    'learning_rate': hp.loguniform('learning_rate', -3, 0),    #lr search space will be in range [exp(-1), exp(0)]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),   #similarly others
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',  #objective is regression object since it is a regression problem
    'seed': 42
}


#after defining the objective fn and search space we need to pass all this information to the fmin method
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest, #algorithm which we willl use for optimization (Tree of Parzen Estimators (TPE))
    max_evals=50,  #number of iterations
    trials=Trials() #trials, this is where all the information about each run wil be stored
)

[16:04:07] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.25256                                                     
[1]	validation-rmse:6.83792                                                     
[2]	validation-rmse:6.67403                                                     
[3]	validation-rmse:6.63856                                                     
[4]	validation-rmse:6.63076                                                     
[5]	validation-rmse:6.62845                                                     
[6]	validation-rmse:6.62240                                                     
[7]	validation-rmse:6.61659                                                     
[8]	validation-rmse:6.60975                                                     
[9]	validation-rmse:6.59957                                                     
[10]	validation-rmse:6.59619                                           

[200]	validation-rmse:6.41900                                                   
[201]	validation-rmse:6.41843                                                   
[202]	validation-rmse:6.41715                                                   
[203]	validation-rmse:6.41630                                                   
[204]	validation-rmse:6.41630                                                   
[205]	validation-rmse:6.41687                                                   
[206]	validation-rmse:6.41599                                                   
[207]	validation-rmse:6.41488                                                   
[208]	validation-rmse:6.41370                                                   
[209]	validation-rmse:6.41414                                                   
[210]	validation-rmse:6.41250                                                   
[211]	validation-rmse:6.41095                                                   
[212]	validation-rmse:6.4106

[9]	validation-rmse:9.48706                                                     
[10]	validation-rmse:9.03442                                                    
[11]	validation-rmse:8.65460                                                    
[12]	validation-rmse:8.33204                                                    
[13]	validation-rmse:8.06524                                                    
[14]	validation-rmse:7.84146                                                    
[15]	validation-rmse:7.65478                                                    
[16]	validation-rmse:7.49821                                                    
[17]	validation-rmse:7.36572                                                    
[18]	validation-rmse:7.25385                                                    
[19]	validation-rmse:7.15878                                                    
[20]	validation-rmse:7.07989                                                    
[21]	validation-rmse:7.01250

[211]	validation-rmse:6.52157                                                   
[212]	validation-rmse:6.52119                                                   
[213]	validation-rmse:6.52096                                                   
[214]	validation-rmse:6.52074                                                   
[215]	validation-rmse:6.52041                                                   
[216]	validation-rmse:6.52023                                                   
[217]	validation-rmse:6.51981                                                   
[218]	validation-rmse:6.51957                                                   
[219]	validation-rmse:6.51940                                                   
[220]	validation-rmse:6.51902                                                   
[221]	validation-rmse:6.51868                                                   
[222]	validation-rmse:6.51819                                                   
[223]	validation-rmse:6.5181

[413]	validation-rmse:6.47094                                                   
[414]	validation-rmse:6.47106                                                   
[415]	validation-rmse:6.47105                                                   
[416]	validation-rmse:6.47043                                                   
[417]	validation-rmse:6.47027                                                   
[418]	validation-rmse:6.46996                                                   
[419]	validation-rmse:6.46995                                                   
[420]	validation-rmse:6.46977                                                   
[421]	validation-rmse:6.46971                                                   
[422]	validation-rmse:6.46947                                                   
[423]	validation-rmse:6.46926                                                   
[424]	validation-rmse:6.46916                                                   
[425]	validation-rmse:6.4688

...

[670]	validation-rmse:6.46788                                                   
[671]	validation-rmse:6.46765                                                   
[672]	validation-rmse:6.46752                                                   
[673]	validation-rmse:6.46657                                                   
[674]	validation-rmse:6.46642                                                   
[675]	validation-rmse:6.46608                                                   
[676]	validation-rmse:6.46612                                                   
[677]	validation-rmse:6.46596                                                   
[678]	validation-rmse:6.46585                                                   
[679]	validation-rmse:6.46551                                                   
[680]	validation-rmse:6.46522                                                   
[681]	validation-rmse:6.46507                                                   
[682]	validation-rmse:6.4649

[872]	validation-rmse:6.43683                                                   
[873]	validation-rmse:6.43676                                                   
[874]	validation-rmse:6.43653                                                   
[875]	validation-rmse:6.43641                                                   
[876]	validation-rmse:6.43631                                                   
[877]	validation-rmse:6.43619                                                   
[878]	validation-rmse:6.43604                                                   
[879]	validation-rmse:6.43590                                                   
[880]	validation-rmse:6.43570                                                   
[881]	validation-rmse:6.43568                                                   
[882]	validation-rmse:6.43553                                                   
[883]	validation-rmse:6.43543                                                   
[884]	validation-rmse:6.4353

[72]	validation-rmse:6.72911                                                    
[73]	validation-rmse:6.72739                                                    
[74]	validation-rmse:6.72620                                                    
[75]	validation-rmse:6.72459                                                    
[76]	validation-rmse:6.72315                                                    
[77]	validation-rmse:6.72196                                                    
[78]	validation-rmse:6.72082                                                    
[79]	validation-rmse:6.71963                                                    
[80]	validation-rmse:6.71877                                                    
[81]	validation-rmse:6.71782                                                    
[82]	validation-rmse:6.71690                                                    
[83]	validation-rmse:6.71630                                                    
[84]	validation-rmse:6.71555

[274]	validation-rmse:6.70027                                                   
[275]	validation-rmse:6.70025                                                   
[276]	validation-rmse:6.70026                                                   
[277]	validation-rmse:6.70032                                                   
[278]	validation-rmse:6.70032                                                   
[279]	validation-rmse:6.70031                                                   
[280]	validation-rmse:6.70025                                                   
[281]	validation-rmse:6.70022                                                   
[282]	validation-rmse:6.70027                                                   
[283]	validation-rmse:6.70032                                                   
[284]	validation-rmse:6.70017                                                   
[285]	validation-rmse:6.70023                                                   
[286]	validation-rmse:6.7001

KeyboardInterrupt: 

In [16]:
#disable autologing so that mlflow cannot any parameters automatically
mlflow.xgboost.autolog(disable=True)

In [15]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    #save the model preprocessor
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    #save the model, this method will check the model and the artifact path
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:45:28] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	validation-rmse:6.82111
[29]	validation-rmse:6.78450
[30]	valida

[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38585
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38510
[282]	validation-rmse:6.38480
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38429
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38187
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38073
[296]	validation-rmse:6.38059
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37986
[301]	validation-rmse:6.37969
[302]	validation-rmse:6.37924
[303]	validation-rmse:6.37890
[304]	validation-rmse:6.37895
[305]	validation-rmse:6.37869
[306]	vali

[547]	validation-rmse:6.33680
[548]	validation-rmse:6.33659
[549]	validation-rmse:6.33672
[550]	validation-rmse:6.33653
[551]	validation-rmse:6.33657
[552]	validation-rmse:6.33651
[553]	validation-rmse:6.33631
[554]	validation-rmse:6.33623
[555]	validation-rmse:6.33601
[556]	validation-rmse:6.33609
[557]	validation-rmse:6.33599
[558]	validation-rmse:6.33582
[559]	validation-rmse:6.33582
[560]	validation-rmse:6.33561
[561]	validation-rmse:6.33555
[562]	validation-rmse:6.33554
[563]	validation-rmse:6.33540
[564]	validation-rmse:6.33525
[565]	validation-rmse:6.33519
[566]	validation-rmse:6.33517
[567]	validation-rmse:6.33522
[568]	validation-rmse:6.33518
[569]	validation-rmse:6.33510
[570]	validation-rmse:6.33491
[571]	validation-rmse:6.33472
[572]	validation-rmse:6.33458
[573]	validation-rmse:6.33443
[574]	validation-rmse:6.33427
[575]	validation-rmse:6.33401
[576]	validation-rmse:6.33395
[577]	validation-rmse:6.33392
[578]	validation-rmse:6.33378
[579]	validation-rmse:6.33362
[580]	vali

[821]	validation-rmse:6.31328
[822]	validation-rmse:6.31317
[823]	validation-rmse:6.31322
[824]	validation-rmse:6.31317
[825]	validation-rmse:6.31318
[826]	validation-rmse:6.31317
[827]	validation-rmse:6.31317
[828]	validation-rmse:6.31309
[829]	validation-rmse:6.31290
[830]	validation-rmse:6.31278
[831]	validation-rmse:6.31271
[832]	validation-rmse:6.31266
[833]	validation-rmse:6.31270
[834]	validation-rmse:6.31273
[835]	validation-rmse:6.31271
[836]	validation-rmse:6.31268
[837]	validation-rmse:6.31260
[838]	validation-rmse:6.31253
[839]	validation-rmse:6.31239
[840]	validation-rmse:6.31230
[841]	validation-rmse:6.31227
[842]	validation-rmse:6.31215
[843]	validation-rmse:6.31216
[844]	validation-rmse:6.31220
[845]	validation-rmse:6.31214
[846]	validation-rmse:6.31200
[847]	validation-rmse:6.31178
[848]	validation-rmse:6.31167
[849]	validation-rmse:6.31149
[850]	validation-rmse:6.31131
[851]	validation-rmse:6.31137
[852]	validation-rmse:6.31137
[853]	validation-rmse:6.31140
[854]	vali

/home/iiitd/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [17]:
#load the save model and predict on a pandas dataframe

#uri
logged_model = 'runs:/7338b6a818bf4893b4ccc0887aab1148/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


[11:59:20] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 7338b6a818bf4893b4ccc0887aab1148

In [19]:
#instead of this, we can also load the model as xgboost model
xgboost_model = mlflow.xgboost.load_model('runs:/7338b6a818bf4893b4ccc0887aab1148/models_mlflow')

[12:12:54] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
xgboost_model

In [23]:

# Predict on a Pandas DataFrame.
y_pred =xgboost_model.predict(valid)

In [24]:
y_pred[:10]

array([14.940359 ,  7.1591063, 14.152481 , 24.3301   ,  9.4101   ,
       17.095747 , 11.566424 ,  8.471772 ,  8.940067 , 20.337719 ],
      dtype=float32)

In [25]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/iiitd/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
